# Import packages

In [1]:
import os
import json
import numpy as np
from sympy import symbols, sympify, lambdify
from tabulate import tabulate
import yaml
from datetime import datetime
import glob

import tensorflow as tf

tf.keras.backend.set_floatx('float64')

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense


# Set random seeds for reproducibility
seed_value = 2023
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

## Allow to print actual tf values
# tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)

2023-12-07 15:00:01.883040: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-07 15:00:01.884857: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 15:00:01.922180: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 15:00:01.922215: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 15:00:01.922243: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

# Constants and parameters

In [2]:
tf_precision = tf.float64
np_precision = np.float64

with open('config_run.yml', 'r') as ymlfile:
    config_data = yaml.load(ymlfile, Loader=yaml.FullLoader)

In [3]:
problem_name = config_data['problem_folder']
EQS_FILE = problem_name + '.json'
EQS_PATH = os.path.join(os.getcwd(), problem_name, EQS_FILE)

training_steps = config_data['epochs']
display_step = training_steps // 20

load_initial_estimate = config_data['load_initial_estimate']
save_best_solution = config_data['save_best_solution']

if len(list(config_data['learning_rate'].keys())) > 1:
    lr_steps = [int(training_steps * key / 100) for key in list(config_data['learning_rate'].keys()) if key != 0]
    lr_values = [float(val) for val in list(config_data['learning_rate'].values())]
    learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay(lr_steps, lr_values)
else:
    learning_rate = float(list(config_data['learning_rate'].values())[0])

In [4]:
# Read the JSON data from the file
with open(EQS_PATH, 'r') as file:
    data = json.load(file)

variables = data['variables']
equations = data['equations']
# equation_terms = data['equation_terms']

variables_str = ' '.join(variables)
variables_sym = symbols(variables_str)
equations_sym = [sympify(eq) for eq in equations]  

## Identify the equation terms
equation_terms = []
for (i, eq) in enumerate(equations_sym):
    terms = []
    for term in eq.args:
        terms.append(str(term))
    equation_terms.append(terms)

# Initialize a list to store the coefficients
scalar_coefficients = []
functions = []
bias_coefficients = []

# Loop through the equation terms
for terms in equation_terms:
    bias_coefficients.append(float(terms[0]))
    coefficients = []   
    for term in terms[1:]:
        # Split each term by the first '*'
        term_parts = term.split('*')
        # Extract the scalar coefficient or default to '1'
        scalar_coeff = term_parts[0] if len(term_parts) > 1 and term_parts[0] not in variables else '1.0'
        coefficients.append(float(scalar_coeff))
               
        # Remove first occurrence of scalar coefficient from the term
        func_coeff = term.replace(term_parts[0]+'*', '', 1) if len(term_parts) > 1 and term_parts[0] not in variables else term
        functions.append(func_coeff)
        
    scalar_coefficients.append(coefficients)   
       
functions_sym = [sympify(func) for func in functions]       
        
print(bias_coefficients)
print(scalar_coefficients)
print(functions)
print(functions_sym)

[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
[[1.0, 1.0, 1.0, 1.0], [2.0, 2.0, 2.0], [6.0, 6.0], [3.0, 3.0, 3.0], [24.0], [12.0, 12.0], [8.0, 8.0], [4.0, 4.0, 4.0]]
['b_1', 'b_2', 'b_3', 'b_4', 'a_21*b_2', 'b_3*(a_31 + a_32)', 'b_4*(a_41 + a_42 + a_43)', 'b_4*(a_21*a_42 + a_43*(a_31 + a_32))', 'a_21*a_32*b_3', 'a_21**2*b_2', 'b_3*(a_31 + a_32)**2', 'b_4*(a_41 + a_42 + a_43)**2', 'a_21*a_32*a_43*b_4', 'b_4*(a_21**2*a_42 + a_43*(a_31 + a_32)**2)', 'a_21**2*a_32*b_3', 'b_4*(a_21*a_42 + a_43*(a_31 + a_32))*(a_41 + a_42 + a_43)', 'a_21*a_32*b_3*(a_31 + a_32)', 'a_21**3*b_2', 'b_3*(a_31 + a_32)**3', 'b_4*(a_41 + a_42 + a_43)**3']
[b_1, b_2, b_3, b_4, a_21*b_2, b_3*(a_31 + a_32), b_4*(a_41 + a_42 + a_43), b_4*(a_21*a_42 + a_43*(a_31 + a_32)), a_21*a_32*b_3, a_21**2*b_2, b_3*(a_31 + a_32)**2, b_4*(a_41 + a_42 + a_43)**2, a_21*a_32*a_43*b_4, b_4*(a_21**2*a_42 + a_43*(a_31 + a_32)**2), a_21**2*a_32*b_3, b_4*(a_21*a_42 + a_43*(a_31 + a_32))*(a_41 + a_42 + a_43), a_21*a_32*b_3*(a_31 + a_32), a

# Functions

In [5]:
def load_most_recent_estimate(problem_name, prefix="best_estimate"):
    # Find all JSON files with the specified prefix
    pattern = os.path.join(problem_name, f"{prefix}_*.json")
    files = glob.glob(pattern)

    if not files:
        print(f"No files found with prefix '{prefix}'")
        return None

    # Get the most recent file based on modification time
    most_recent_file = max(files, key=os.path.getmtime)
    
    print(f"Loading most recent estimate from {most_recent_file}")

    # Load json file with initial estimate (best from the most recent run)
    with open(most_recent_file, 'r') as file:
        data = json.load(file)

    return data['best_solution']


@tf.autograph.experimental.do_not_convert
def evaluate_tf_function(inputs, values, symbolic_function):
    # Ensure that the number of inputs and values match
    if len(inputs) != len(values):
        raise ValueError("Number of inputs and values must match.")

    # Evaluate the symbolic function using TensorFlow and the provided values
    result = symbolic_function(*values)
    # Convert the result to a TensorFlow tensor
    result = tf.convert_to_tensor(result, dtype=tf_precision)
    
    # print(result.__class__)
    
    return result


# Create model
def mlp_model(num_outputs, hidden_layers = [16, 16]):

    # # Reshape input if necessary, matching the shape of the first layer's weights
    # x = tf.reshape(x, [1, -1])  # Adjust the shape as needed
    
    mlp_input = Input(shape=(1,1), name="mlp_input")
    
    hidden_layer = mlp_input
    
    for li, layer_size in enumerate(hidden_layers):
        hidden_layer = Dense(layer_size, activation='tanh', name=f"hidden_layer_{li+1}")(hidden_layer)

    output_layer = Dense(num_outputs, activation='relu', name="output_layer")(hidden_layer)
    
    return Model(inputs=mlp_input, outputs=output_layer)


@tf.autograph.experimental.do_not_convert
def customMSE(vars=variables_sym, funcs=functions_sym):
    
    def mse(y_true, y_pred):
        
        y_pred_values = y_pred[0, 0, :]
        
        var_vals = [tf.squeeze(y_pred[0, 0, i]) for i in range(y_pred_values.shape[0])]
        
        # print(f"var_vals = {var_vals}")
        
        nn_residual = list()
        for func in funcs:
            func_tf = lambdify(vars, func, 'tensorflow')
            nn_residual.append(evaluate_tf_function(variables_sym, var_vals, func_tf))
            
        print(f"nn_residual = {nn_residual}")
        
        nn_residual_tf = tf.convert_to_tensor(nn_residual, dtype=tf_precision)
        
        # print(f"nn_residual_tf = {nn_residual_tf}")
        
        mse_value = tf.reduce_mean(tf.square(nn_residual_tf))
        
        # print(f"mse_value = {mse_value}")   
        
        return mse_value
        
        # return y_pred

    return mse


# Model

## Create and compile model

In [6]:
# Network Parameters
num_input = 1 # input layer

num_vars = len(variables)
num_output = num_vars

# Create the model
model = mlp_model(num_output, hidden_layers = [8, 8])
model.summary()

# Define the optimizer and loss function
optimizer = Adam(learning_rate=learning_rate)
loss_object = customMSE(variables_sym, functions_sym)

# Compile the model
model.compile(optimizer=optimizer, loss=loss_object, metrics=['mse'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mlp_input (InputLayer)      [(None, 1, 1)]            0         
                                                                 
 hidden_layer_1 (Dense)      (None, 1, 8)              16        
                                                                 
 hidden_layer_2 (Dense)      (None, 1, 8)              72        
                                                                 
 output_layer (Dense)        (None, 1, 10)             90        
                                                                 
Total params: 178 (1.39 KB)
Trainable params: 178 (1.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
num_vars

10

## Train the model

In [8]:
# Create a constant input (e.g., zeros) since the input is supposed to be constant
constant_input = np.ones((1, 1, 1))

# Assuming num_output is the number of output nodes in your model
# You can use a dummy value for y_true since your loss function doesn't depend on it
dummy_y_true = np.zeros((1, num_output))

# Fit the model
model.fit(x=constant_input, y=dummy_y_true, 
          epochs=training_steps, 
          batch_size=1,
          verbose=2)


Epoch 1/101


/media/jesus/GSUS-DATA/CANADA_docs/RaymondSpiteri_USASK_SK/RKTK_equation_learning/nn-non-linear-systems/venv-nn-systems/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
nn_residual = [<tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.5266269347291972>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.11157275434110989>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Te

## Test the model

In [ ]:
# Test the model
model.predict(constant_input)

1/1 [==============================] - 0s 231ms/step


array([[[0.07171434, 0.40059327, 0.        , 0.47509958, 0.1603949 ,
         0.        , 0.        , 0.        , 0.        , 0.        ]]])